<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEkphrasisnegpos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

Mounted at /content/drive
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...

In [2]:
df_train.head(1000)

,Tweet,Emotion
3667,o job interview today I am realli go to mess t...,2
11298,agesthegreat ii do not have ani either,2
10975,so freak tire i want my sleep pattern to be no...,2
10794,elliott claim steak and shake ha been do thi f...,2
3361,one of these day I am go to delet someth reall...,2
...,...,...
9247,say goodmorn,1
9434,i should be in nyc right now with some of my f...,2
8439,hate the everyth about seafood just so happen ...,2
1944,whi telstra whi do you are biggest teleco have...,2


In [3]:
df_test.Emotion.unique()

array(['0', '2', '1', '3', 'emotion'], dtype=object)

In [4]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [5]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [6]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [7]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(1303215, 1724630)

In [8]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [9]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [10]:
positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])
pn_dict = {}
for i in range(len(positive)):
  pn_dict[positive[i]] = 1

for i in range(len(negative)):
  pn_dict[negative[i]] = -1

In [11]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
X_train = [nltk.pos_tag(x) for x in X_train]
X_test = [nltk.pos_tag(x) for x in X_test]
X_train = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_train]
X_test = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_test]
result_table = [0,0,0,0,0,0,0,0,0]

In [12]:
print(X_train[0][0])

('o', 'JJ', 1)


In [13]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.get_vector(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [14]:
epochs = 20

In [15]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
24/24 [==============================] - 5s 225ms/step - loss: 1.2638 - accuracy: 0.5713
Epoch 2/20
24/24 [==============================] - 6s 235ms/step - loss: 1.0361 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9872 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 6s 240ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 6s 241ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 6s 235ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 10/20
24/24 [=====

In [16]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/20
24/24 [==============================] - 6s 231ms/step - loss: 1.2183 - accuracy: 0.5708
Epoch 2/20
24/24 [==============================] - 6s 247ms/step - loss: 1.0012 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 240ms/step - loss: 0.9842 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 239ms/step - loss: 0.9835 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 234ms/step - loss: 0.9832 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 5s 228ms/step - loss: 0.9831 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 5s 228ms/step - loss: 0.9829 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 6s 230ms/step - loss: 0.9828 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 231ms/step - loss: 0.9826 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 6s 232ms/step - loss: 0.9824 - accuracy: 0.5863

In [17]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/20
24/24 [==============================] - 6s 248ms/step - loss: 1.1229 - accuracy: 0.5687
Epoch 2/20
24/24 [==============================] - 6s 253ms/step - loss: 0.9871 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 254ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 249ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 253ms/step - loss: 0.9837 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 6s 261ms/step - loss: 0.9837 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 6s 253ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 6s 247ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 250ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 6s 255ms/step - loss: 0.9836 - accuracy: 0.5863

In [18]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

Epoch 1/20
24/24 [==============================] - 6s 234ms/step - loss: 1.2409 - accuracy: 0.5728
Epoch 2/20
24/24 [==============================] - 6s 233ms/step - loss: 1.0075 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 235ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 6s 234ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 6s 235ms/step - loss: 0.9835 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 237ms/step - loss: 0.9835 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 6s 240ms/step - loss: 0.9835 - accuracy: 0.5863

In [19]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

Epoch 1/20
24/24 [==============================] - 6s 232ms/step - loss: 1.2166 - accuracy: 0.5710
Epoch 2/20
24/24 [==============================] - 6s 236ms/step - loss: 1.0030 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 239ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 239ms/step - loss: 0.9834 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9832 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9831 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 6s 232ms/step - loss: 0.9829 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 6s 234ms/step - loss: 0.9827 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 235ms/step - loss: 0.9826 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9823 - accuracy: 0.5863

In [20]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

Epoch 1/20
24/24 [==============================] - 6s 247ms/step - loss: 1.1128 - accuracy: 0.5674
Epoch 2/20
24/24 [==============================] - 6s 245ms/step - loss: 0.9859 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 245ms/step - loss: 0.9842 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 247ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 245ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 6s 242ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 6s 245ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 6s 243ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 246ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 6s 249ms/step - loss: 0.9840 - accuracy: 0.5863

In [21]:
epochs=40
Embedding_size=100

In [22]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


Epoch 1/40
24/24 [==============================] - 1s 51ms/step - loss: 1.2059 - accuracy: 0.5411
Epoch 2/40
24/24 [==============================] - 1s 52ms/step - loss: 0.9851 - accuracy: 0.5863
Epoch 3/40
24/24 [==============================] - 1s 53ms/step - loss: 0.9749 - accuracy: 0.5864
Epoch 4/40
24/24 [==============================] - 1s 52ms/step - loss: 0.9724 - accuracy: 0.5864
Epoch 5/40
24/24 [==============================] - 1s 51ms/step - loss: 0.9706 - accuracy: 0.5864
Epoch 6/40
24/24 [==============================] - 1s 50ms/step - loss: 0.9678 - accuracy: 0.5864
Epoch 7/40
24/24 [==============================] - 1s 53ms/step - loss: 0.9620 - accuracy: 0.5864
Epoch 8/40
24/24 [==============================] - 1s 50ms/step - loss: 0.9541 - accuracy: 0.5867
Epoch 9/40
24/24 [==============================] - 1s 50ms/step - loss: 0.9459 - accuracy: 0.5885
Epoch 10/40
24/24 [==============================] - 1s 51ms/step - loss: 0.9378 - accuracy: 0.5925
Epoch 11/

In [23]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

Epoch 1/40
24/24 [==============================] - 1s 53ms/step - loss: 1.0794 - accuracy: 0.5713
Epoch 2/40
24/24 [==============================] - 1s 54ms/step - loss: 0.9637 - accuracy: 0.5867
Epoch 3/40
24/24 [==============================] - 1s 54ms/step - loss: 0.9443 - accuracy: 0.5891
Epoch 4/40
24/24 [==============================] - 1s 53ms/step - loss: 0.9285 - accuracy: 0.5914
Epoch 5/40
24/24 [==============================] - 1s 53ms/step - loss: 0.9111 - accuracy: 0.5993
Epoch 6/40
24/24 [==============================] - 1s 53ms/step - loss: 0.8953 - accuracy: 0.6063
Epoch 7/40
24/24 [==============================] - 1s 53ms/step - loss: 0.8823 - accuracy: 0.6146
Epoch 8/40
24/24 [==============================] - 1s 54ms/step - loss: 0.8713 - accuracy: 0.6212
Epoch 9/40
24/24 [==============================] - 1s 54ms/step - loss: 0.8599 - accuracy: 0.6258
Epoch 10/40
24/24 [==============================] - 1s 53ms/step - loss: 0.8483 - accuracy: 0.6332
Epoch 11/

In [24]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/40
24/24 [==============================] - 2s 67ms/step - loss: 1.1080 - accuracy: 0.5210
Epoch 2/40
24/24 [==============================] - 2s 66ms/step - loss: 0.9901 - accuracy: 0.5853
Epoch 3/40
24/24 [==============================] - 2s 65ms/step - loss: 0.9878 - accuracy: 0.5857
Epoch 4/40
24/24 [==============================] - 2s 66ms/step - loss: 0.9876 - accuracy: 0.5857
Epoch 5/40
24/24 [==============================] - 2s 65ms/step - loss: 0.9875 - accuracy: 0.5857
Epoch 6/40
24/24 [==============================] - 2s 64ms/step - loss: 0.9867 - accuracy: 0.5858
Epoch 7/40
24/24 [==============================] - 2s 65ms/step - loss: 0.9860 - accuracy: 0.5860
Epoch 8/40
24/24 [==============================] - 2s 65ms/step - loss: 0.9857 - accuracy: 0.5859
Epoch 9/40
24/24 [==============================] - 2s 65ms/step - loss: 0.9844 - accuracy: 0.5860
Epoch 10/40
24/24 [==============================] - 2s 67ms/step - loss: 0.9816 - accuracy: 0.5862
Epoch 11/

In [25]:
model_sg1 = tf.keras.Sequential()
model_sg1.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg1.add(tf.keras.layers.LSTM(64))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg1.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg1.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg1.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")

Epoch 1/40
24/24 [==============================] - 6s 230ms/step - loss: 1.2186 - accuracy: 0.5771
Epoch 2/40
24/24 [==============================] - 6s 241ms/step - loss: 1.0215 - accuracy: 0.5858
Epoch 3/40
24/24 [==============================] - 6s 238ms/step - loss: 1.0042 - accuracy: 0.5857
Epoch 4/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9958 - accuracy: 0.5863
Epoch 5/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9933 - accuracy: 0.5863
Epoch 6/40
24/24 [==============================] - 6s 236ms/step - loss: 0.9941 - accuracy: 0.5863
Epoch 7/40
24/24 [==============================] - 6s 236ms/step - loss: 0.9916 - accuracy: 0.5863
Epoch 8/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9918 - accuracy: 0.5863
Epoch 9/40
24/24 [==============================] - 6s 233ms/step - loss: 0.9917 - accuracy: 0.5863
Epoch 10/40
24/24 [==============================] - 6s 235ms/step - loss: 0.9907 - accuracy: 0.5863

In [26]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.5845000147819519, 0.5845000147819519, 0.5845000147819519]
cbow [0.5845000147819519, 0.5845000147819519, 0.5845000147819519]
glove [0.750333309173584, 0.7973333597183228, 0.6434166431427002]


In [27]:
print(result_table[9])

0.5845000147819519
